In [27]:
import squigglepy as sq
import numpy as np
import pandas as pd
from squigglepy.numbers import K, M, B

sq.set_seed(42)
np.random.seed(42)
np.seterr(invalid='raise')  # Warn on operations involving NaN
N_SAMPLES = 5000

from chip_estimates_utils import (
    normalize_shares,
    estimate_cumulative_chip_sales,
    aggregate_by_chip_type,
    compute_running_totals,
)

In [28]:
# ==============================================
# GOOGLE SHEETS CONFIGURATION
# ==============================================

SPREADSHEET_ID = "1RfjL98gsSe9QeXDnBOraticvS-s13XmKvHv_dwwa3W8"

# Sheet GIDs
REVENUE_GID = 0
PROD_MIX_GID = 141507805

REVENUE_URL = f"https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/gviz/tq?tqx=out:csv&gid={REVENUE_GID}"
PROD_MIX_URL = f"https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/gviz/tq?tqx=out:csv&gid={PROD_MIX_GID}"

In [29]:
# ==============================================
# LOAD DATA FROM GOOGLE SHEETS
# ==============================================

revenue_df = pd.read_csv(REVENUE_URL)
revenue_df = revenue_df[['quarter', 'instinct_revenue_p5', 'instinct_revenue_p95']].dropna()

prod_mix_df = pd.read_csv(PROD_MIX_URL)
prod_mix_df = prod_mix_df[['quarter', 'chip_type', 'share_p5', 'share_p95']].dropna()

QUARTERS = revenue_df['quarter'].tolist()
AMD_VERSIONS = prod_mix_df['chip_type'].dropna().unique().tolist()

print(f"Loaded {len(revenue_df)} quarters of revenue data")
print(revenue_df)
print()
print(f"Loaded {len(prod_mix_df)} version-quarter combinations")
print(prod_mix_df)

Loaded 7 quarters of revenue data
   quarter  instinct_revenue_p5  instinct_revenue_p95
0  FY24 Q1                  0.6                  0.90
1  FY24 Q2                  1.0                  1.25
2  FY24 Q3                  1.4                  1.60
3  FY24 Q4                  1.7                  1.90
4  FY25 Q1                  1.3                  1.80
5  FY25 Q2                  0.8                  1.30
6  FY25 Q3                  1.7                  2.40

Loaded 22 version-quarter combinations
    quarter chip_type  share_p5  share_p95
0   FY24 Q1    MI300A      0.01       0.05
1   FY24 Q1    MI250X      0.25       0.50
2   FY24 Q1    MI300X      0.45       0.60
3   FY24 Q2    MI300X      0.75       0.90
4   FY24 Q2    MI300A      0.02       0.10
5   FY24 Q2    MI250X      0.01       0.10
6   FY24 Q3    MI300X      0.70       0.90
7   FY24 Q3    MI300A      0.15       0.25
8   FY24 Q3    MI250X      0.01       0.05
9   FY24 Q4    MI300X      0.75       0.85
10  FY24 Q4    MI300A

In [30]:
# ==============================================
# CONVERT TO SQUIGGLEPY DISTRIBUTIONS
# ==============================================

# Convert revenue DataFrame to dictionary of distributions (revenue in billions)
AMD_REVENUE = {}
for _, row in revenue_df.iterrows():
    quarter = row['quarter']
    AMD_REVENUE[quarter] = sq.norm(row['instinct_revenue_p5'], row['instinct_revenue_p95'])

# Convert production mix DataFrame to nested dictionary of distributions
PROD_MIX = {}
for quarter in prod_mix_df['quarter'].unique():
    quarter_data = prod_mix_df[prod_mix_df['quarter'] == quarter]
    PROD_MIX[quarter] = {}
    for _, row in quarter_data.iterrows():
        chip_type = row['chip_type']
        PROD_MIX[quarter][chip_type] = sq.norm(row['share_p5'], row['share_p95'], lclip=0, rclip=1)

# Average Selling Prices (ASP)
AMD_PRICES = {
    'MI300X': sq.to(10000, 15000),
    'MI300A': sq.to(6000, 7500),
    'MI325X': sq.to(10000, 20000),
    'MI250X': sq.to(8000, 12000),
    'MI350X': sq.to(22000, 25000),
    'MI355X': sq.to(25000, 30000),
}

print("AMD prices (90% CI):")
for v, price in AMD_PRICES.items():
    print(f"  {v}: ${price.x:,.0f}-${price.y:,.0f}")

AMD prices (90% CI):
  MI300X: $10,000-$15,000
  MI300A: $6,000-$7,500
  MI325X: $10,000-$20,000
  MI250X: $8,000-$12,000
  MI350X: $22,000-$25,000
  MI355X: $25,000-$30,000


In [31]:
# ==============================================
# CHIP SPECS FROM EPOCH.AI
# ==============================================
# Download chip specs including TOPS and TDP

import requests
import zipfile
import io

# Map notebook AMD chip names to epoch.ai CSV names (CSV uses "Instinct " prefix)
AMD_NAME_MAP = {
    'MI250X': 'Instinct MI250X',
    'MI300X': 'Instinct MI300X',
    'MI300A': 'Instinct MI300A',
    'MI325X': 'Instinct MI325X',
    'MI350X': 'Instinct MI350X',
    'MI355X': 'Instinct MI355X',
}

# Fallback specs (TOPS from 8-bit OP/s, TDP in watts)
FALLBACK_SPECS = {
    'MI250X': {'tops': 383,  'tdp': 500},
    'MI300X': {'tops': 2615, 'tdp': 750},
    'MI300A': {'tops': 1961, 'tdp': 760},
    'MI325X': {'tops': 2615, 'tdp': 1000},
    'MI350X': {'tops': 4614, 'tdp': 1000},
    'MI355X': {'tops': 5033, 'tdp': 1400},
}

H100_TOPS = 1979  # Reference for H100-equivalent calculation

def load_amd_specs(chip_types_df, name_map, fallback_specs):
    """Extract TOPS and TDP from CSV, with fallbacks for missing values."""
    specs = {}
    for notebook_name, csv_name in name_map.items():
        row = chip_types_df[chip_types_df['Name'] == csv_name]
        if len(row) == 1:
            # Parse 8-bit OP/s
            tops_raw = row['8-bit OP/s'].values[0]
            tops = float(tops_raw) / 1e12 if pd.notna(tops_raw) else fallback_specs[notebook_name]['tops']
            
            # Parse TDP
            tdp_col = 'TDP (W) (from ML Hardware (linked))'
            tdp_raw = row[tdp_col].values[0] if tdp_col in row.columns else None
            tdp = float(tdp_raw) if pd.notna(tdp_raw) else fallback_specs[notebook_name]['tdp']
            
            specs[notebook_name] = {'tops': tops, 'tdp': tdp}
        else:
            specs[notebook_name] = fallback_specs[notebook_name].copy()
            print(f"Warning: Using fallback specs for {notebook_name} (not found in CSV)")
    return specs

# Try to download and parse chip specs from epoch.ai
try:
    url = "https://epoch.ai/data/ai_chip_sales.zip"
    response = requests.get(url, timeout=10)
    response.raise_for_status()
    
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        with z.open("chip_types.csv") as f:
            chip_types_df = pd.read_csv(f)
            amd_chips_df = chip_types_df[chip_types_df['Designer'] == 'AMD']
    
    print(amd_chips_df[['Name', 'TDP (W) (from ML Hardware (linked))', '8-bit OP/s']].head(10))
    AMD_SPECS = load_amd_specs(amd_chips_df, AMD_NAME_MAP, FALLBACK_SPECS)
    print("\nLoaded AMD specs from epoch.ai CSV:")
except Exception as e:
    print(f"Warning: Could not download chip specs from epoch.ai: {e}")
    print("Using fallback AMD specs:")
    AMD_SPECS = FALLBACK_SPECS.copy()

for chip, spec in AMD_SPECS.items():
    print(f"  {chip}: TOPS={spec['tops']:.0f}, TDP={spec['tdp']:.0f}W")

               Name  TDP (W) (from ML Hardware (linked))        8-bit OP/s
4   Instinct MI250X                                500.0   383000000000000
9   Instinct MI300A                                760.0  1961200000000000
14  Instinct MI308X                                750.0   653700000000000
15  Instinct MI300X                                750.0  2614900000000000
17  Instinct MI325X                               1000.0  2600000000000000
22  Instinct MI350X                               1000.0  4614000000000000
23  Instinct MI355X                               1400.0  5033200000000000

Loaded AMD specs from epoch.ai CSV:
  MI250X: TOPS=383, TDP=500W
  MI300X: TOPS=2615, TDP=750W
  MI300A: TOPS=1961, TDP=760W
  MI325X: TOPS=2600, TDP=1000W
  MI350X: TOPS=4614, TDP=1000W
  MI355X: TOPS=5033, TDP=1400W


In [32]:
# ==============================================
# SAMPLING FUNCTIONS
# ==============================================

def sample_revenue(quarter):
    """Sample quarterly revenue in dollars."""
    return (AMD_REVENUE[quarter] @ 1) * B


def sample_shares(quarter):
    """Sample and normalize production mix shares for a quarter."""
    mix = PROD_MIX[quarter]
    raw_shares = {chip_type: dist @ 1 for chip_type, dist in mix.items()}
    return normalize_shares(raw_shares)


def sample_base_price(chip_type):
    """Sample base price for a chip type (called once per chip, reused across quarters)."""
    return AMD_PRICES[chip_type] @ 1


# No deflation factor needed for AMD - prices are direct ASPs
def get_deflation_factor(quarter, chip_type):
    """Return price adjustment factor. AMD uses direct ASPs, so always 1.0."""
    return 1.0

In [33]:
# ==============================================
# RUN CORRELATED SIMULATION
# ==============================================

quarterly_samples = estimate_cumulative_chip_sales(
    quarters=QUARTERS,
    chip_types=AMD_VERSIONS,
    sample_revenue=sample_revenue,
    sample_shares=sample_shares,
    sample_base_price=sample_base_price,
    get_deflation_factor=get_deflation_factor,
    sample_revenue_uncertainty=None,
    n_samples=N_SAMPLES,
)

cumulative_samples = aggregate_by_chip_type(quarterly_samples)

print("Simulation complete.")

Simulation complete.


In [34]:
# ==============================================
# CUMULATIVE SUMMARY
# ==============================================

def print_cumulative_summary(cumulative_samples, versions, title="Cumulative Production"):
    print(f"{title}")
    print(f"{'Version':<8} {'p5':>12} {'p50':>12} {'p95':>12}")
    print("-" * 51)

    grand_total = None
    for v in versions:
        arr = cumulative_samples[v]
        if arr.sum() > 0:
            if grand_total is None:
                grand_total = np.zeros_like(arr)
            grand_total += arr
            print(f"{v:<8} {int(np.percentile(arr, 5)):>12,} {int(np.percentile(arr, 50)):>12,} {int(np.percentile(arr, 95)):>12,}")

    if grand_total is not None:
        print("-" * 51)
        print(f"{'TOTAL':<8} {int(np.percentile(grand_total, 5)):>12,} {int(np.percentile(grand_total, 50)):>12,} {int(np.percentile(grand_total, 95)):>12,}")

print_cumulative_summary(cumulative_samples, AMD_VERSIONS, "Cumulative AMD Instinct Production (Correlated Model)")

Cumulative AMD Instinct Production (Correlated Model)
Version            p5          p50          p95
---------------------------------------------------
MI300A         94,212      114,813      137,640
MI250X         31,856       45,019       61,428
MI300X        405,800      506,181      621,419
MI325X         51,742       79,286      120,157
MI350X         31,159       40,344       51,375
MI355X          8,219       12,399       17,566
---------------------------------------------------
TOTAL         691,142      802,204      924,770


In [35]:
# ==============================================
# CUMULATIVE RUNNING TOTALS BY QUARTER
# ==============================================
# Note: AMD uses calendar quarters, so no interpolation needed

running_totals_samples = compute_running_totals(quarterly_samples)

print("Cumulative Running Totals by Quarter")
print(f"{'Quarter':<10} {'Version':<8} {'p5':>12} {'p50':>12} {'p95':>12}")
print("=" * 60)

for quarter in QUARTERS:
    quarter_has_data = False
    for v in AMD_VERSIONS:
        arr = running_totals_samples[quarter][v]
        if arr.sum() > 0:
            quarter_has_data = True
            print(f"{quarter:<10} {v:<8} {int(np.percentile(arr, 5)):>12,} {int(np.percentile(arr, 50)):>12,} {int(np.percentile(arr, 95)):>12,}")

    if quarter_has_data:
        total = sum(running_totals_samples[quarter][v] for v in AMD_VERSIONS)
        print(f"{quarter:<10} {'TOTAL':<8} {int(np.percentile(total, 5)):>12,} {int(np.percentile(total, 50)):>12,} {int(np.percentile(total, 95)):>12,}")
        print("-" * 60)

Cumulative Running Totals by Quarter
Quarter    Version            p5          p50          p95
FY24 Q1    MI300A          1,119        3,562        6,350
FY24 Q1    MI250X         20,550       30,360       42,918
FY24 Q1    MI300X         24,888       34,466       47,968
FY24 Q1    TOTAL          53,545       69,186       87,643
------------------------------------------------------------
FY24 Q2    MI300A          6,917       14,405       22,114
FY24 Q2    MI250X         25,294       36,989       51,546
FY24 Q2    MI300X         90,911      115,528      145,662
FY24 Q2    TOTAL         139,189      167,616      201,665
------------------------------------------------------------
FY24 Q3    MI300A         44,152       57,693       72,337
FY24 Q3    MI250X         28,620       41,496       56,980
FY24 Q3    MI300X        168,245      211,004      261,732
FY24 Q3    TOTAL         266,042      310,990      363,220
------------------------------------------------------------
FY24 Q4    MI

In [ ]:
# ==============================================
# CSV EXPORTS: CUMULATIVE RUNNING TOTALS BY CHIP TYPE
# ==============================================
# Export running totals broken down by AMD chip type
# Note: AMD uses calendar-aligned fiscal quarters, mapping FY24 Q1 -> Q1 2024, etc.

def get_amd_quarter_dates(quarter):
    """Return (start_date, end_date) strings for AMD fiscal quarter like 'FY24 Q1'.
    AMD fiscal quarters align with calendar quarters (FY24 Q1 = Jan-Mar 2024)."""
    parts = quarter.split()
    fy = parts[0]  # e.g., 'FY24'
    q_num = int(parts[1][1])  # e.g., 1 from 'Q1'
    year = 2000 + int(fy[2:])  # e.g., 2024 from 'FY24'
    
    if q_num == 1:
        return f"1/1/{year}", f"3/31/{year}"
    elif q_num == 2:
        return f"4/1/{year}", f"6/30/{year}"
    elif q_num == 3:
        return f"7/1/{year}", f"9/30/{year}"
    else:
        return f"10/1/{year}", f"12/31/{year}"

def get_calendar_quarter_label(quarter):
    """Convert AMD fiscal quarter to calendar quarter label."""
    parts = quarter.split()
    fy = parts[0]  # e.g., 'FY24'
    q_part = parts[1]  # e.g., 'Q1'
    year = 2000 + int(fy[2:])  # e.g., 2024 from 'FY24'
    return f"{q_part} {year}"

# Get first quarter dynamically
first_quarter = QUARTERS[0]
first_calendar_quarter = get_calendar_quarter_label(first_quarter)
first_start_date, _ = get_amd_quarter_dates(first_quarter)

rows = []
for quarter in QUARTERS:
    _, end_date = get_amd_quarter_dates(quarter)
    cq_label = get_calendar_quarter_label(quarter)
    for chip in AMD_VERSIONS:
        arr = running_totals_samples[quarter][chip]
        if arr.sum() > 0:
            rows.append({
                'Name': f"{chip} {first_calendar_quarter} to {cq_label}",
                'Start date': first_start_date,
                'End date': end_date,
                'Chip type': chip,
                'Number of units (5th percentile)': int(np.percentile(arr, 5)),
                'Number of units (median)': int(np.percentile(arr, 50)),
                'Number of units (95th percentile)': int(np.percentile(arr, 95)),
            })

by_chip_df = pd.DataFrame(rows)
by_chip_df.to_csv('amd_cumulative_by_chip.csv', index=False)
print(f"Exported {len(by_chip_df)} rows to amd_cumulative_by_chip.csv")
# print(by_chip_df.head(10))

Exported 27 rows to amd_cumulative_by_chip.csv
                        Name Start date    End date Chip type  \
0  MI300A Q1 2024 to Q1 2024   1/1/2024   3/31/2024    MI300A   
1  MI250X Q1 2024 to Q1 2024   1/1/2024   3/31/2024    MI250X   
2  MI300X Q1 2024 to Q1 2024   1/1/2024   3/31/2024    MI300X   
3  MI300A Q1 2024 to Q2 2024   1/1/2024   6/30/2024    MI300A   
4  MI250X Q1 2024 to Q2 2024   1/1/2024   6/30/2024    MI250X   
5  MI300X Q1 2024 to Q2 2024   1/1/2024   6/30/2024    MI300X   
6  MI300A Q1 2024 to Q3 2024   1/1/2024   9/30/2024    MI300A   
7  MI250X Q1 2024 to Q3 2024   1/1/2024   9/30/2024    MI250X   
8  MI300X Q1 2024 to Q3 2024   1/1/2024   9/30/2024    MI300X   
9  MI300A Q1 2024 to Q4 2024   1/1/2024  12/31/2024    MI300A   

   Number of units (5th percentile)  Number of units (median)  \
0                              1119                      3562   
1                             20550                     30360   
2                             24888       

In [ ]:
# ==============================================
# CSV EXPORTS: FULL-AMD AGGREGATE STATS
# ==============================================
# Export running totals with aggregate metrics across all AMD chip types
# Metrics: total units, H100e compute, power (MW)

# Get first quarter dynamically
first_quarter = QUARTERS[0]
first_calendar_quarter = get_calendar_quarter_label(first_quarter)
first_start_date, _ = get_amd_quarter_dates(first_quarter)

rows = []
for quarter in QUARTERS:
    _, end_date = get_amd_quarter_dates(quarter)
    cq_label = get_calendar_quarter_label(quarter)
    
    # Compute aggregate metrics across all AMD chip types
    n_samples = N_SAMPLES
    units_total = np.zeros(n_samples)
    h100e_total = np.zeros(n_samples)
    power_total_w = np.zeros(n_samples)
    
    for chip in AMD_VERSIONS:
        arr = running_totals_samples[quarter][chip]
        units_total += arr
        if chip in AMD_SPECS:
            tops = AMD_SPECS[chip]['tops']
            tdp = AMD_SPECS[chip]['tdp']
            h100e_total += arr * (tops / H100_TOPS)
            power_total_w += arr * tdp
    
    rows.append({
        'Name': f"AMD total {first_calendar_quarter} to {cq_label}",
        'Designer': 'AMD',
        'Start date': first_start_date,
        'End date': end_date,
        'Number of units (5th percentile)': int(np.percentile(units_total, 5)),
        'Number of units (median)': int(np.percentile(units_total, 50)),
        'Number of units (95th percentile)': int(np.percentile(units_total, 95)),
        'Compute estimate in H100e (5th percentile)': int(np.percentile(h100e_total, 5)),
        'Compute estimate in H100e (median)': int(np.percentile(h100e_total, 50)),
        'Compute estimate in H100e (95th percentile)': int(np.percentile(h100e_total, 95)),
        'Power in MW (5th percentile)': int(np.percentile(power_total_w / 1e6, 5)),
        'Power in MW (median)': int(np.percentile(power_total_w / 1e6, 50)),
        'Power in MW (95th percentile)': int(np.percentile(power_total_w / 1e6, 95)),
    })

totals_df = pd.DataFrame(rows)
totals_df.to_csv('amd_cumulative_totals.csv', index=False)
print(f"Exported {len(totals_df)} rows to amd_cumulative_totals.csv")
# print(totals_df)

Exported 7 rows to amd_cumulative_totals.csv
                           Name Designer Start date    End date  \
0  AMD total Q1 2024 to Q1 2024      AMD   1/1/2024   3/31/2024   
1  AMD total Q1 2024 to Q2 2024      AMD   1/1/2024   6/30/2024   
2  AMD total Q1 2024 to Q3 2024      AMD   1/1/2024   9/30/2024   
3  AMD total Q1 2024 to Q4 2024      AMD   1/1/2024  12/31/2024   
4  AMD total Q1 2024 to Q1 2025      AMD   1/1/2024   3/31/2025   
5  AMD total Q1 2024 to Q2 2025      AMD   1/1/2024   6/30/2025   
6  AMD total Q1 2024 to Q3 2025      AMD   1/1/2024   9/30/2025   

   Number of units (5th percentile)  Number of units (median)  \
0                             53545                     69186   
1                            139189                    167616   
2                            266042                    310990   
3                            415570                    483078   
4                            523316                    606996   
5                           

In [38]:
# ==============================================
# UNCORRELATED MODEL COMPARISON
# ==============================================

from chip_estimates_utils import estimate_chip_sales

def sample_price(quarter, chip_type):
    """Sample price independently for each quarter (uncorrelated)."""
    return AMD_PRICES[chip_type] @ 1

uncorrelated_samples = estimate_chip_sales(
    quarters=QUARTERS,
    versions=AMD_VERSIONS,
    sample_revenue=sample_revenue,
    sample_shares=sample_shares,
    sample_price=sample_price,
    n_samples=1000,  # smaller for speed
)

cumulative_uncorrelated_samples = aggregate_by_chip_type(uncorrelated_samples)

In [39]:
print_cumulative_summary(cumulative_uncorrelated_samples, AMD_VERSIONS, "Cumulative AMD Instinct Production (Uncorrelated Model)")
print(" ")
print_cumulative_summary(cumulative_samples, AMD_VERSIONS, "Cumulative AMD Instinct Production (Correlated Model)")

Cumulative AMD Instinct Production (Uncorrelated Model)
Version            p5          p50          p95
---------------------------------------------------
MI300A         96,440      115,257      134,901
MI250X         33,763       45,419       59,386
MI300X        456,193      507,493      561,743
MI325X         57,388       81,332      112,096
MI350X         31,490       40,687       51,545
MI355X          8,078       12,470       17,263
---------------------------------------------------
TOTAL         743,490      805,072      871,080
 
Cumulative AMD Instinct Production (Correlated Model)
Version            p5          p50          p95
---------------------------------------------------
MI300A         94,212      114,813      137,640
MI250X         31,856       45,019       61,428
MI300X        405,800      506,181      621,419
MI325X         51,742       79,286      120,157
MI350X         31,159       40,344       51,375
MI355X          8,219       12,399       17,566
------------